In [1]:
import requests
from pydantic import BaseModel
from typing import List, Literal
from geopy.geocoders import Nominatim
import folium
from folium import Marker, Circle, Icon
import re
from folium.plugins import MarkerCluster

In [2]:
class Ambulatorio(BaseModel) :
    orario: str 
    telefono: str 
    segreteria: str
    comune: str
    ubicazione: str
    principale: Literal["Y", "N"]


class Doc(BaseModel) :
     nome: str 
     cognome: str
     tipologia: Literal["PLS","MMG"]
     ambito: str
     aft: str
     codice_regionale: str
     scelte_disponibili: int
     id: str
     ambulatori : List[Ambulatorio]
     
class Location(BaseModel) :
    latitude : float
    longitude : float

In [9]:
url_prova = "https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&azienda=1813960"
url_prova 

'https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&azienda=1813960'

In [3]:
url_base ="https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&tipologia=pls&azienda=1813961&comune=ALTOPASCIO"

In [6]:
def make_call_medici_asl_toscana(tipo_medico : str, asl :str, comune : str) -> list | None :
    dict_nome_to_cod_asl = {"asl_centro" : "1813960" ,
                            "asl_nord" : "1813961",
                            "asl_sud" : "1813962"}
    url_call = f"https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&tipologia={tipo_medico}&azienda={dict_nome_to_cod_asl[asl]}&comune={comune}"
    response = requests.get(url_call)
    medici = None
    if response.status_code == 200 :
        meidici=  response.json["medici"]
    return medici

In [4]:
req  = requests.get(url_base)
req

<Response [200]>

In [5]:
req.status_code

200

In [11]:
json_req = req.json()
json_req["medici"]

[{'nome': 'CARMELA',
  'cognome': 'ABELA',
  'tipologia': 'MMG',
  'ambito': 'FIRENZE - GEN',
  'aft': 'CAMPO DI MARTE',
  'codice_regionale': '010165',
  'scelte_disponibili': '1108',
  'id': '10010665',
  'ambulatori': [{'orario': 'Lun : 16:00 - 17:00(A); Mer : 15:30 - 16:30(A); Ven : 10:00 - 11:00(A); ',
    'telefono': '3206978229',
    'segreteria': '3206978229   3206978229',
    'comune': 'FIRENZE',
    'ubicazione': 'VIA SANTO STEFANO IN PANE, 4/A, FIRENZE',
    'principale': 'N'},
   {'orario': 'Lun : 18:30 - 19:30(A); Mar : 09:00 - 11:00(A); Gio : 09:00 - 10:00(A); ',
    'telefono': '3206978229',
    'segreteria': '3206978229   3206978229',
    'comune': 'FIRENZE',
    'ubicazione': 'VIA DI CAMERATA, 10, FIRENZE',
    'principale': 'N'}]},
 {'nome': 'FIAMMA',
  'cognome': 'ABRAMI',
  'tipologia': 'MMG',
  'ambito': 'DICOMANO-LONDA-SAN GODENZO - GEN',
  'aft': 'MUGELLO EST',
  'codice_regionale': '000455',
  'scelte_disponibili': '122',
  'id': '10017279',
  'ambulatori': [{'o

In [12]:
def get_pediatri(lista_medici : list,)  -> List[Doc] :
    pediatri = []
    for medico in lista_medici :
        #print(medico["tipologia"])
        if medico["tipologia"] == "PLS" :
            pediatri.append(Doc(**medico))
    return pediatri
     

In [13]:
pediatri = get_pediatri(json_req["medici"])

In [20]:
pediatri[0]

Doc(nome='MOHAMMAD', cognome='AL-OMARI', tipologia='PLS', ambito='FIRENZE - PED', aft='AFT TIPO MEDICO 2 PER IMPORTAZIONE CENTRO', codice_regionale='002766', scelte_disponibili=2, id='10010706', ambulatori=[Ambulatorio(orario='Lun Mer Gio : 15:30 - 19:30(A); Mar Ven : 10:00 - 13:00(A); ', telefono='055570017', segreteria='055570017', comune='FIRENZE', ubicazione='VIA BORGHINI, 1, FIRENZE', principale='Y'), Ambulatorio(orario='Lun Mer Gio : 08:00 - 12:00; Mar Ven : 08:00 - 10:00; 14:30 - 16:30; ', telefono='366-3775315', segreteria='366-3775315', comune='FIRENZE', ubicazione="CONTATTABILITA' TELEFONICA, FIRENZE", principale='N')])

In [147]:
def check_valid_word(word :str,) :
    if len(word) <= 1 :
        if word.isdigit() :
            return True
        else :
            return False
    return True
    

In [158]:
def simplify_address(address : str, pattern : str = "[^a-zA-Z0-9/]+") :
    cancellatore = re.compile(pattern)
    clean_address = re.sub(cancellatore, " ", address)
    new_address= ""
    for word in clean_address.split() :
        if check_valid_word(word) :
            new_address += word + " "
    return new_address.strip(" ") 

In [9]:
def get_lat_log_from_address(address :str, geolocator : object) -> Location :
    location = geolocator.geocode(address)
    if location is None :
        location = geolocator.geocode(simplify_address(address))
    #print(location)
    lock_return  = None
    if location is None :
        print(f"sorry could not locate {address}")
    else :
        lock_return = Location(latitude=location.latitude, longitude=location.longitude)
    return lock_return

In [26]:
def set_icon_color(value : int) :
    if value > 10 :
        return "green"
    if (value <= 10) and (value > 0) :
        return "orange"
    if value  <= 0 :
        return "red"

In [160]:
def make_popup_str(doc : Doc, ambulatorio : Ambulatorio) -> str :
    pop_up_string = f"{doc.nome} {doc.cognome}\n posti disponibili: {doc.scelte_disponibili} \n orario : {ambulatorio.orario} \n telefono : {ambulatorio.telefono}\n segreteria: {ambulatorio.segreteria}  \nindirizzo: {ambulatorio.ubicazione}"
    return pop_up_string

In [5]:
set_icon_color(11)

'green'

In [14]:
def add_marker_to_map(lista_medici :List[Doc], mappa_base : folium.Map) :
    geocoder = Nominatim(user_agent="asl_ricerca",country_codes="IT")
    marker_cluster = MarkerCluster().add_to(mappa_base)
    for medici in lista_medici :
        lista_unbicazioni = medici.ambulatori
        for location in lista_unbicazioni :
            #controlla che non sia la contattabilità telefonica
            if ("contattabilit" not in location.ubicazione.lower()) and ("per prenotazion" not in location.ubicazione.lower()) and ("per richiest" not in location.ubicazione.lower()):
                #print(location)
                
                location_geo = get_lat_log_from_address(location.ubicazione, geocoder)
                #print(location_geo)
                if location_geo is None :
                    print(f"could not locate {location.ubicazione} of{medici.nome} {medici.cognome}")
                if location_geo is not None :
                    icon = Icon(icon="house-medical",  color = set_icon_color(medici.scelte_disponibili) , prefix="fa")
                    merk = Marker([location_geo.latitude, location_geo.longitude],popup=make_popup_str(medici, location),
                                  icon=icon).add_to(marker_cluster)
    return mappa_base   
    

In [10]:
prova_ambulatorio = json_req["medici"][0]["ambulatori"][0]
prova_ambulatorio

{'orario': 'Lun : 16:00 - 17:00(A); Mer : 15:30 - 16:30(A); Ven : 10:00 - 11:00(A); ',
 'telefono': '3206978229',
 'segreteria': '3206978229   3206978229',
 'comune': 'FIRENZE',
 'ubicazione': 'VIA SANTO STEFANO IN PANE, 4/A, FIRENZE',
 'principale': 'N'}

In [152]:
simplify_address("PIAZZA IV NOVEMBRE 28 A.P., LAMPORECCHIO", "[^a-zA-Z0-9/]+")

'PIAZZA IV NOVEMBRE 28 LAMPORECCHIO'

In [7]:
geolocator = Nominatim(user_agent="asl_ricerca")
location_prova = geolocator.geocode("Lastra a signa",
                                     ,country_codes="IT")
print(location_prova)

Lastra a Signa, Firenze, Toscana, 50055, Italia


In [50]:
location_prova.attribute

AttributeError: 'NoneType' object has no attribute 'attribute'

In [15]:
lat_long_prova= get_lat_log_from_address("lastra a signa", geolocator)
mappa_base = folium.Map(location=[lat_long_prova.latitude, lat_long_prova.longitude], zoom_start=13)
mappa_base

In [20]:
Marker([location_prova.latitude, location_prova.longitude], ).add_to(mappa_base)

In [164]:
mappa_base = add_marker_to_map(pediatri, mappa_base)
mappa_base

sorry could not locate Corso Matteotti, 30, FUCECCHIO
could not locate Corso Matteotti, 30, FUCECCHIO ofPIETRO ARZILLI
sorry could not locate per richesta visite domiciliari, FUCECCHIO
could not locate per richesta visite domiciliari, FUCECCHIO ofPIETRO ARZILLI
sorry could not locate VIA DANTE ALIGHIERI, 10 INT. C - STANZA 5, LASTRA A SIGNA
could not locate VIA DANTE ALIGHIERI, 10 INT. C - STANZA 5, LASTRA A SIGNA ofPAOLO BECHERUCCI
sorry could not locate VIA NALDINI, 44 c/o distretto USL A.P., BARBERINO TAVARNELLE
could not locate VIA NALDINI, 44 c/o distretto USL A.P., BARBERINO TAVARNELLE ofELISA BECUCCI
sorry could not locate VIA C.BATTISTI, 87 C/O CENTRO HELIOS, FUCECCHIO
could not locate VIA C.BATTISTI, 87 C/O CENTRO HELIOS, FUCECCHIO ofGIULIA BINI
sorry could not locate VIA G. D'ANNUNZIO, 29 c/o presidio USL A.P., FIRENZE
could not locate VIA G. D'ANNUNZIO, 29 c/o presidio USL A.P., FIRENZE ofARIANNA BOCELLI
sorry could not locate VIA L. BECHERONI 2 A.P., VERNIO
could not locate

In [21]:
mappa_base